In [ ]:
#Import packages
import os
import flopy
import numpy as np
import matplotlib.pyplot as plt
import flopy.utils.binaryfile as bf
import geopandas as gpd
from scipy.stats import norm

In [ ]:
## Set up directory referencing
# Package data
git_dir = os.path.dirname(os.path.dirname(os.getcwd()))
gwfm_dir = '\\'.join(str.split(git_dir,'\\')[0:3])+ '/Box/research_cosumnes/GWFlowModel'
print(git_dir, gwfm_dir)

In [ ]:
loadpth = 'C:/wrdapp/GWFlowModel/Cosumnes/Cosumnes_simple'
model_ws = loadpth+'/WEL_RIV_RCH_tprogs_200layers/realization000'
m = flopy.modflow.Modflow.load('MF.nam', model_ws=model_ws, 
                                exe_name='MF2005', version='mf2005')

In [ ]:
def plot_lines(lines):
    fig, ax = plt.subplots()
    for ln in lines:
        plt.plot([ln[0][0], ln[1][0]], [ln[0][1], ln[1][1]], linewidth = 0.1)

In [ ]:
##Grid Creation for Large Model
# Load model grid as geopandas object
grid_regional = gpd.read_file(gwfm_dir+'/DIS_data/grid/grid.shp')
# grid_p = gpd.read_file(gwfm_dir+'/DIS_data/44_7_grid/44_7_grid.shp')
grid_p = gpd.read_file(gwfm_dir+'/DIS_data/Inset_grid_44_7/44_7_grid.shp')

grid_p_temp = grid_p.copy()
grid_p_temp.geometry = grid_p_temp.buffer(-99.9999)
# grid_regional.geometry = grid_regional.buffer(-100)

grid_ref = gpd.sjoin(grid_p_temp, grid_regional.to_crs('epsg:3310'), how = 'left',predicate = 'within',
                     lsuffix='inset', rsuffix = 'regional')
# grid_ref = pd.DataFrame(grid_ref.loc[:,['row_inset','column_inset','row_regional','column_regional']])
# reset geometry from tiny polygon to rectangle
grid_ref.geometry = grid_p.geometry

grid_ref = grid_ref.rename({'row_inset':'row','column_inset':'column'},axis=1)

In [ ]:
##Teichert Model Load
Teichert = gpd.read_file('C:/wrdapp/OHWD recharge project files/Kautz Property.shp')
Teichert_3310 = Teichert.to_crs('epsg:3310')

In [ ]:
for n in np.arange(0,100).astype(str):
    model = 'C:/wrdapp/GWFlowModel/Cosumnes/Cosumnes_simple/WEL_RIV_RCH_tprogs_200layers/realization'+ n.zfill(3)+'/'
    hdobj = flopy.utils.HeadFile(model + '/MF.hds')
    head = hdobj.get_data(kstpkper = spd_stp[0])
    head.shape
    A = head[100,:,:]
    A_new = np.reshape(A,(6300,1))
    grid_ref["Head"+n] = A_new


In [ ]:
spd_stp[0]

In [ ]:
##Clip Data
#grid_ref = gpd.sjoin(grid_p_temp, grid_regional.to_crs('epsg:3310'), how = 'left',predicate = 'within',
                     #lsuffix='inset', rsuffix = 'regional')
grid_teichert = gpd.sjoin(grid_ref, Teichert_3310)
#grid_teichert.plot()
grid_teichert

In [ ]:
#grid_teichert.plot()
fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot()
grid_teichert.plot(ax = ax)
B1 = grid_teichert.iloc[80:81,:]
B1.plot(ax = ax, color = 'green')
B2 = grid_teichert.iloc[79:80,:]
B2.plot(ax = ax, color = 'red')
B3 = grid_teichert.iloc[66:67,:]
B3.plot(ax = ax, color = 'orange')
#B4 = grid_teichert.iloc[95:96,:]
#B4.plot(ax = ax, color = 'yellow')
#B5 = grid_teichert.iloc[96:97,:]
#B5.plot(ax = ax, color = 'purple')

In [ ]:
for r in np.arange(0,104):
    Plt = plt.figure(r)
    X = grid_teichert.iloc[r,8:108]    
    #norm.pdf(X)
    Plt = X.plot.kde()
    plt.ylim(0,1.25)
    plt.xlabel('Head Value')

In [ ]:
plt.rcParams.update({'font.size': 16})
Plt = plt.figure()
ax = Plt.add_subplot(1,1,1)
plt.ylim(0,1.3)
X = grid_teichert.iloc[61,8:108]
Plt = X.plot.kde(ax = ax, color = 'green')
X1 = grid_teichert.iloc[74,8:108]
Plt = X1.plot.kde(ax = ax, color = 'red')
X2 = grid_teichert.iloc[75,8:108]
Plt = X2.plot.kde(ax = ax, color = 'orange')
X3 = grid_teichert.iloc[95,8:108]
Plt = X3.plot.kde(ax = ax, color = 'yellow')
X4 = grid_teichert.iloc[96,8:108]
Plt = X4.plot.kde(ax = ax, color = 'purple')
    #Plt = PDF.plot()
    #AAA = find_peaks(PDF)
    #AAA
plt.xlabel('Head Value (m)')
plt.title('Teichert Highest Peaks')

In [ ]:
plt.rcParams.update({'font.size': 16})
Plt = plt.figure()
ax = Plt.add_subplot(1,1,1)
plt.ylim(0,1.3)
X = grid_teichert.iloc[80,8:108]
Plt = X.plot.kde(ax = ax, color = 'green')
X1 = grid_teichert.iloc[79,8:108]
Plt = X1.plot.kde(ax = ax, color = 'red')
X2 = grid_teichert.iloc[66,8:108]
Plt = X2.plot.kde(ax = ax, color ='orange')
#X3 = grid_teichert.iloc[95,8:108]
#Plt = X3.plot.kde(ax = ax)
#X4 = grid_teichert.iloc[96,8:108]
#Plt = X4.plot.kde(ax = ax)
    #Plt = PDF.plot()
    #AAA = find_peaks(PDF)
    #AAA
plt.xlabel('Head Value (m)')
plt.title('Teichert Largest Spread')

In [ ]:
##Plot Head Data
def plot_head(hdobj, name):
    head = hdobj.get_data()
    levels_hmax = np.round(np.max(head[head!=1E+30]),-1)
#     levels_hmin = np.round(np.min(head[head!=-999.99]),-1)
    levels_hmin = np.round(np.min(head[head>=-200]),-1)
   # frf = cbb.get_data(text='FLOW RIGHT FACE')[0]
   # fff = cbb.get_data(text='FLOW FRONT FACE')[0]

    levels = np.arange(levels_hmin, levels_hmax, 2)

    fig = plt.figure(figsize=(30,20))

    ax = fig.add_subplot(1,1,1)
    ax.set_title(name+'Head Contours')
    mapview = flopy.plot.PlotMapView(model=m)
    quadmesh = mapview.plot_ibound()
#     line = mapview.plot_bc('RIV', plotAll=True)
    mapview.plot_bc("GHB", plotAll=True)
    mapview.plot_bc("CHD", plotAll=True)
    Teichert = gpd.read_file('C:/wrdapp/OHWD recharge project files/Kautz Property.shp')
    Teichert_3310 = Teichert.to_crs('epsg:3310')
    Teichert_3310.plot(ax = ax)

    contour_set = mapview.contour_array(head[5,:,:], masked_values=[-999.99, 1e+30], levels=levels)
#     quiver = mapview.plot_discharge(frf, fff, istep=10, jstep=10)  # no head array for volumetric discharge

    cb = plt.colorbar(contour_set, shrink = 0.5)
    cb.set_label('Head (m)')

    ax.ticklabel_format(style='plain')

    linecollection = mapview.plot_grid(linewidths = 0.3)
    plt.xlabel('Easting (m)')
    plt.ylabel('Northing (m)')
#     plt.savefig('Plots//'+name +'_head_contour.png', dpi = 600, bbox_inches='tight')

In [ ]:
##Extract Heads to Array
hdobj = flopy.utils.HeadFile(model_ws + '/MF.hds')
#spd_stp = hdobj.get_kstpkper()
# for i in np.arange(0,len(spd_stp)):
plot_head(hdobj, 'MF')
#spd_stp
plt.savefig('Head_Contours')

In [ ]:
hdobj = flopy.utils.HeadFile(model_ws + '/MF.hds')
#hdobj.list_records()
rec = hdobj.get_data()
len(hdobj)